# ALX-ROBOTICS AI
  This repository contains resources and projects related to Artificial Intelligence (AI) within the ALX Robotics program.

  ## Overview

  ALX Robotics offers a comprehensive curriculum encompassing various AI aspects including:

  * **Machine Learning:** Supervised, unsupervised, and reinforcement learning algorithms.
  * **Deep Learning:** Neural networks, convolutional neural networks (CNNs), recurrent neural networks (RNNs).
  * **Computer Vision:** Image processing, object detection, and image classification.
  * **Natural Language Processing (NLP):** Text analysis, language modeling, and sentiment analysis.
  * **Robotics:** Integration of AI algorithms into robotic systems for autonomous navigation and manipulation.

  ## Structure

  The repository might include the following folders and files:

  * **Projects:** Contains individual or group project folders, showcasing implemented AI solutions.
  * **Datasets:** Contains pre-processed or raw data used for training AI models.
  * **Documentation:** Provides information about the project, including design choices and implementation details.
  * **Tutorials:** Contains example code and notebooks for different AI concepts and tools.
  * **Libraries:** Contains custom Python libraries developed for the project.

  ## Contributing

  We welcome contributions from students, educators, and researchers.

  **To contribute:**

  1. Fork the repository.
  2. Create a new branch for your contribution.
  3. Commit your changes and push them to your branch.
  4. Submit a pull request to the main branch.

  ## License

  This repository is licensed under the MIT License.


  ## Contact

  For any questions or inquiries, please contact [Your contact information].

In [ ]:
!pip install pandas matplotlib requests

# ML PREDICTION & ANOMALIES DETECTION

In [ ]:
'''
* DATA MONITORING & VISUALIZATION------------------
*  # Devices:
*     -> DHT22 sensor
*     -> ESP32
*  # Technologies:
*     -> Protocol: HTTP
*     -> Platform: ThingSpeak + Google Colab
*  # Activity:
*    -> Data transmission, monitoring and visualization
*       - Single chart on Google Colab
*
*   ANGAZA ELIMU&ALX - IOT SCHOOL: Cohort 1, 2024
* --------------------------------------------------
'''

'''
 Support libraries -------------------------------------------------------------
'''
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import userdata

#---- ThingSpeak API details
CHANNEL_ID = userdata.get('CHANNEL_ID')
API_KEY = userdata.get('API_KEY') # Read API Key
THINGSPEAK_URL = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={API_KEY}&results=100"

# ---- Fetch data from ThingSpeak
response = requests.get(THINGSPEAK_URL)
data = response.json()

#---- Parse data
timestamps = [entry['created_at'] for entry in data['feeds']]
temperature = [float(entry['field1']) for entry in data['feeds']]
humidity = [float(entry['field2']) for entry in data['feeds']]

#---- Create DataFrame
df = df.dropna()
df = pd.DataFrame({'Timestamp': timestamps, 'Temperature': temperature, 'Humidity': humidity})
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

#---- Plot data
plt.figure(figsize=(10,5))
plt.plot(df['Timestamp'], df['Temperature'], label='Temperature (°C)')
plt.plot(df['Timestamp'], df['Humidity'], label='Humidity (%)')
plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.title('Temperature and Humidity Monitoring')
plt.legend()
plt.show()


# ML PREDICTION & ANOMALIES DETECTION

In [ ]:
'''
* ML PREDICTION & ANOMALIES DETECTION ------------------
*  # Devices:
*     -> DHT22 sensor
*     -> ESP32
*  # Technologies:
*     -> Protocol: HTTP
*     -> Platform: ThingSpeak + Google Colab
*     -> ML Tools: Prediction & Regression - LinearRegression
*  # Activity:
*    -> Using ML for data processing, prediction and anomalies detection
*
*   ANGAZA ELIMU&ALX - IOT SCHOOL: Cohort 1, 2024
* --------------------------------------------------
'''

# Import necessary libraries
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# ThingSpeak Configuration
CHANNEL_ID = userdata.get('CHANNEL_ID')
READ_API_KEY = userdata.get('API_KEY') # Read API Key

# Section 1: Data Collection from ThingSpeak
def fetch_data():
    url = f'https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={READ_API_KEY}&results=1000'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['feeds'])
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['temperature'] = pd.to_numeric(df['field1'], errors='coerce')
    df['humidity'] = pd.to_numeric(df['field2'], errors='coerce')
    print("Data fetched successfully.")
    return df[['created_at', 'temperature', 'humidity']]

# Section 2: Data Preparation
def prepare_data(df):
    df['time_index'] = (df['created_at'] - df['created_at'].min()).dt.total_seconds()
    X = df[['time_index']]
    y_temp = df['temperature']
    y_hum = df['humidity']
    print("Data prepared successfully.")
    return X, y_temp, y_hum

# Section 3: Train Temperature and Humidity Prediction Models
def train_regression_models(X, y_temp, y_hum):
    # Split data for training and testing
    X_train, X_test, y_temp_train, y_temp_test = train_test_split(X, y_temp, test_size=0.2, random_state=42)
    _, _, y_hum_train, y_hum_test = train_test_split(X, y_hum, test_size=0.2, random_state=42)

    # Train temperature prediction model
    temp_model = LinearRegression()
    temp_model.fit(X_train, y_temp_train)
    temp_predictions = temp_model.predict(X_test)
    temp_mse = mean_squared_error(y_temp_test, temp_predictions)

    # Train humidity prediction model
    hum_model = LinearRegression()
    hum_model.fit(X_train, y_hum_train)
    hum_predictions = hum_model.predict(X_test)
    hum_mse = mean_squared_error(y_hum_test, hum_predictions)

    print(f"Temperature Prediction MSE: {temp_mse}")
    print(f"Humidity Prediction MSE: {hum_mse}")
    return temp_model, hum_model

# Section 4.1: Anomaly Detection for Humidity
def humid_anomaly_detection(df):
    hum_mean, hum_std = df['humidity'].mean(), df['humidity'].std()
    threshold_upper = hum_mean + 2 * hum_std
    threshold_lower = hum_mean - 2 * hum_std
    df['humidity_anomaly'] = df['humidity'].apply(lambda x: 'Anomaly' if x > threshold_upper or x < threshold_lower else 'Normal')
    print("Anomaly detection completed.")
    return threshold_upper, threshold_lower

# Section 4.2: Anomaly Detection for Temperature
def temp_anomaly_detection(df):
    temp_mean, temp_std = df['temperature'].mean(), df['temperature'].std()
    threshold_upper = temp_mean + 2 * temp_std
    threshold_lower = temp_mean - 2 * temp_std
    df['temperature_anomaly'] = df['temperature'].apply(lambda x: 'Anomaly' if x > threshold_upper or x < threshold_lower else 'Normal')
    print("Anomaly detection completed.")
    return threshold_upper, threshold_lower

# Section 5: Predict and Test on New Data
def test_new_data(temp_model, hum_model, temp_thresholds, hum_thresholds):
    # Fetch new data
    new_data = fetch_data()
    new_data['time_index'] = (new_data['created_at'] - new_data['created_at'].min()).dt.total_seconds()
    X_new = new_data[['time_index']]

    # Make predictions
    new_data['predicted_temperature'] = temp_model.predict(X_new)
    new_data['predicted_humidity'] = hum_model.predict(X_new)

    # Apply anomaly detection for humidity
    hum_upper, hum_lower = hum_thresholds
    new_data['humidity_anomaly'] = new_data['humidity'].apply(lambda x: 'Anomaly' if x > hum_upper or x < hum_lower else 'Normal')

    # Apply anomaly detection for temperature
    temp_upper, temp_lower = temp_thresholds
    new_data['temperature_anomaly'] = new_data['temperature'].apply(lambda x: 'Anomaly' if x > temp_upper or x < temp_lower else 'Normal')

    print("Predictions and anomaly detection on new data:")
    print(new_data[['created_at', 'temperature', 'predicted_temperature', 'temperature_anomaly',
                    'humidity', 'predicted_humidity', 'humidity_anomaly']].tail())

# Main Execution Flow
if __name__ == "__main__":
    # Step 1: Fetch and prepare the data
    df = fetch_data()
    X, y_temp, y_hum = prepare_data(df)

    # Step 2: Train the ML models for temperature and humidity
    temp_model, hum_model = train_regression_models(X, y_temp, y_hum)

    # Step 3: Set up and apply anomaly detection for humidity and temperature
    temp_thresholds = temp_anomaly_detection(df)
    hum_thresholds = humid_anomaly_detection(df)

    # Step 4: Test and use the models on new data
    test_new_data(temp_model, hum_model, temp_thresholds, hum_thresholds)


# ML CLASSIFICATION OF DATA TRENDS

In [ ]:
'''
* ML CLASSIFICATION OF DATA TRENDS ------------------
*  # Devices:
*     -> DHT22 sensor
*     -> ESP32
*  # Technologies:
*     -> Protocol: HTTP
*     -> Platform: ThingSpeak + Google Colab
*     -> ML Tools: Data trends classification - RandomForestClassifier
*  # Activity:
*    -> Building a classification model to classify temperature trends (e.g., "Rising", "Falling", or "Stable")
*       based on the dataset collected from a DHT22 sensor.
*
*   ANGAZA ELIMU&ALX - IOT SCHOOL: Cohort 1, 2024
* --------------------------------------------------
'''

# Import necessary libraries
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# ThingSpeak Configuration
CHANNEL_ID = userdata.get('CHANNEL_ID')
READ_API_KEY = userdata.get('API_KEY') # Read API Key

# Section 1: Data Collection from ThingSpeak
def fetch_data():
    """
    Fetches temperature and humidity data from a ThingSpeak channel.
    Returns a DataFrame with timestamp, temperature, and humidity columns.
    """
    url = f'https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={READ_API_KEY}&results=1000'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['feeds'])
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['temperature'] = pd.to_numeric(df['field1'], errors='coerce')
    df['humidity'] = pd.to_numeric(df['field2'], errors='coerce')
    print("Data fetched successfully.")
    return df[['created_at', 'temperature', 'humidity']]

# Section 2: Preprocess and label data for trends
def label_temperature_trends(df):
    """
    Label temperature trends:
    - Rising: if temperature at t > temperature at t-1
    - Falling: if temperature at t < temperature at t-1
    - Stable: if temperature at t == temperature at t-1
    """
    df['temperature_diff'] = df['temperature'].diff()
    df['trend'] = df['temperature_diff'].apply(
        lambda x: 'Rising' if x > 0 else ('Falling' if x < 0 else 'Stable')
    )
    df = df.dropna()  # Drop the first row since it will have NaN for diff
    print("Temperature trends labeled successfully.")
    return df

# Section 3: Split data into features and labels
def prepare_data(df):
    # Features: Use time-based index and previous temperature values
    df['time_index'] = (df['created_at'] - df['created_at'].min()).dt.total_seconds()
    X = df[['time_index', 'temperature']]
    y = df['trend']
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Section 4: Train a classification model
def train_classification_model(X_train, y_train):
    clf = RandomForestClassifier(random_state=42, n_estimators=100)
    clf.fit(X_train, y_train)
    print("Classification model trained successfully.")
    return clf

# Section 5: Test and evaluate the model
def evaluate_model(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    return y_pred

# Section 6: Visualize classification results
def visualize_results(df, y_pred, X_test):
    df_test = pd.DataFrame(X_test)
    df_test['predicted_trend'] = y_pred
    plt.figure(figsize=(10, 6))
    for trend in df_test['predicted_trend'].unique():
        subset = df_test[df_test['predicted_trend'] == trend]
        plt.scatter(subset['time_index'], subset['temperature'], label=trend)
    plt.xlabel("Time Index")
    plt.ylabel("Temperature")
    plt.title("Temperature Trend Classification")
    plt.legend()
    plt.show()

# Section 7: Predict and classify trends on new data
def classify_new_data(clf):
    new_data = fetch_data()
    new_data['time_index'] = (new_data['created_at'] - new_data['created_at'].min()).dt.total_seconds()
    X_new = new_data[['time_index', 'temperature']]
    new_data['predicted_trend'] = clf.predict(X_new)
    print("Predicted trends on new data:")
    print(new_data[['created_at', 'temperature', 'predicted_trend']].tail())
    return new_data

# Main execution flow
if __name__ == "__main__":
    # Step 1: Fetch and preprocess the data
    df = fetch_data()
    df = label_temperature_trends(df)

    # Step 2: Split the data
    X_train, X_test, y_train, y_test = prepare_data(df)

    # Step 3: Train the classification model
    clf = train_classification_model(X_train, y_train)

    # Step 4: Evaluate the model
    y_pred = evaluate_model(clf, X_test, y_test)

    # Step 5: Visualize classification results
    visualize_results(df, y_pred, X_test)

    # Step 6: Predict on new data
    classify_new_data(clf)


In [ ]:
'''
* DATA MONITORING & VISUALIZATION------------------
*  # Devices:
*     -> DHT22 sensor
*     -> ESP32
*  # Technologies:
*     -> Protocol: HTTP
*     -> Platform: ThingSpeak + Google Colab
*  # Activity:
*    -> Data transmission, monitoring and visualization
*       - Anomalies detection in temperature&humidity data trends
*
*   ANGAZA ELIMU&ALX - IOT SCHOOL: Cohort 1, 2024
* --------------------------------------------------
'''

'''
 Support libraries -------------------------------------------------------------
'''
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt

# ---- ThingSpeak API details
CHANNEL_ID = userdata.get('CHANNEL_ID')
READ_API_KEY = userdata.get('API_KEY') # Read API Key
THINGSPEAK_URL = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={API_KEY}&results=100"

# ---- Fetch data from ThingSpeak
response = requests.get(THINGSPEAK_URL)
data = response.json()

# ---- Extract data
timestamps = [entry['created_at'] for entry in data['feeds']]
temperature = [float(entry['field1']) for entry in data['feeds']]
humidity = [float(entry['field2']) for entry in data['feeds']]

# ---- Dataframe setup
df = df.dropna()
df = pd.DataFrame({'Timestamp': timestamps, 'Temperature': temperature, 'Humidity': humidity})
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# ---- Anomaly thresholds definition
# compute the average mean and std deviation
temp_mean, temp_std = np.mean(df['Temperature']), np.std(df['Temperature'])
hum_mean, hum_std = np.mean(df['Humidity']), np.std(df['Humidity'])

# define lower and upper limit thresholds
temp_threshold_upper = temp_mean + 2 * temp_std
temp_threshold_lower = temp_mean - 2 * temp_std
hum_threshold_upper = hum_mean + 2 * hum_std
hum_threshold_lower = hum_mean - 2 * hum_std

# ---- Flag anomalies
df['Temp_Anomaly'] = (df['Temperature'] > temp_threshold_upper) | (df['Temperature'] < temp_threshold_lower)
df['Hum_Anomaly'] = (df['Humidity'] > hum_threshold_upper) | (df['Humidity'] < hum_threshold_lower)

# ---- Plot data and anomalies
plt.figure(figsize=(12, 8))

# ---- Plot temperature data with mean and threshold lines
plt.subplot(2, 1, 1)
plt.plot(df['Timestamp'], df['Temperature'], label='Temperature', color='blue')
plt.axhline(temp_mean, color='green', linestyle='--', label='Temperature Mean')
plt.axhline(temp_threshold_upper, color='red', linestyle='--', label='Temperature Upper Threshold')
plt.axhline(temp_threshold_lower, color='orange', linestyle='--', label='Temperature Lower Threshold')
plt.scatter(df[df['Temp_Anomaly']]['Timestamp'], df[df['Temp_Anomaly']]['Temperature'], color='red', label='Temperature Anomaly', s=50, zorder=5)
plt.fill_between(df['Timestamp'], temp_threshold_lower, temp_threshold_upper, color='lightblue', alpha=0.1, label='Normal Temperature Range')
plt.xlabel('Timestamp')
plt.ylabel('Temperature (°C)')
plt.legend(loc='upper left')
plt.title('Anomaly Detection in Temperature')

# ---- Plot humidity data with mean and threshold lines
plt.subplot(2, 1, 2)
plt.plot(df['Timestamp'], df['Humidity'], label='Humidity', color='purple')
plt.axhline(hum_mean, color='green', linestyle='--', label='Humidity Mean')
plt.axhline(hum_threshold_upper, color='red', linestyle='--', label='Humidity Upper Threshold')
plt.axhline(hum_threshold_lower, color='orange', linestyle='--', label='Humidity Lower Threshold')
plt.scatter(df[df['Hum_Anomaly']]['Timestamp'], df[df['Hum_Anomaly']]['Humidity'], color='purple', label='Humidity Anomaly', s=50, zorder=5)
plt.fill_between(df['Timestamp'], hum_threshold_lower, hum_threshold_upper, color='lightgreen', alpha=0.1, label='Normal Humidity Range')
plt.xlabel('Timestamp')
plt.ylabel('Humidity (%)')
plt.legend(loc='upper left')
plt.title('Anomaly Detection in Humidity')

plt.tight_layout()
plt.show()


# DATA MONITORING & VISUALIZATION | PLOT

In [ ]:
'''
* DATA MONITORING & VISUALIZATION------------------
*  # Devices:
*     -> DHT22 sensor
*     -> ESP32
*  # Technologies:
*     -> Protocol: HTTP
*     -> Platform: ThingSpeak + Google Colab
*  # Activity:
*    -> Data transmission, monitoring and visualization
*       - Time series basic weather forecasting and prediction
*
*   ANGAZA ELIMU&ALX - IOT SCHOOL: Cohort 1, 2024
* --------------------------------------------------
'''

'''
 Support libraries -------------------------------------------------------------
'''
import requests
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet

# ---- ThingSpeak API details
CHANNEL_ID = userdata.get('CHANNEL_ID')
READ_API_KEY = userdata.get('API_KEY') # Read API Key
THINGSPEAK_URL = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={API_KEY}&results=100"

# ---- Fetch data from ThingSpeak
response = requests.get(THINGSPEAK_URL)
data = response.json()

# ---- Parse data
timestamps = [entry['created_at'] for entry in data['feeds']]
temperature = [float(entry['field1']) for entry in data['feeds']]
humidity = [float(entry['field2']) for entry in data['feeds']]

# ---- Create DataFrame
df = df.dropna()
df = pd.DataFrame({'Timestamp': timestamps, 'Temperature': temperature, 'Humidity': humidity})
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# ---- Prepare data for Prophet - the future forecasting handler lib
temperature_df = df[['Timestamp', 'Temperature']].rename(columns={'Timestamp': 'ds', 'Temperature': 'y'})
humidity_df = df[['Timestamp', 'Humidity']].rename(columns={'Timestamp': 'ds', 'Humidity': 'y'})

# ---- Remove timezone from 'ds' column - timezone not required for the prophet future forecasting lib handler lib
temperature_df['ds'] = temperature_df['ds'].dt.tz_localize(None)
humidity_df['ds'] = humidity_df['ds'].dt.tz_localize(None)

# ---- Fit Prophet model for Temperature Forecasting
temp_model = Prophet(interval_width=0.95)  # Set prediction interval to 95%
temp_model.fit(temperature_df)

# ---- Forecasting 24 hours into the future
future_temp = temp_model.make_future_dataframe(periods=24, freq='h')
temp_forecast = temp_model.predict(future_temp)

# ---- Plot Temperature Forecast
plt.figure(figsize=(10, 5))
temp_model.plot(temp_forecast, xlabel='Time', ylabel='Temperature (°C)')
plt.title('24-Hour Temperature Forecast with Historical Trends')
plt.grid(True)

# ---- Highlight observed data period
plt.fill_between(temp_forecast['ds'], temp_forecast['yhat_lower'], temp_forecast['yhat_upper'], color='skyblue', alpha=0.2)
plt.annotate('Future Predicted Range', xy=(temp_forecast['ds'].iloc[-5], temp_forecast['yhat'].iloc[-5]), color='blue')

# ---- Fit Prophet model for Humidity Forecasting
humidity_model = Prophet(interval_width=0.95)
humidity_model.fit(humidity_df)

# ---- Forecasting 24 hours into the future
future_humidity = humidity_model.make_future_dataframe(periods=24, freq='h')
humidity_forecast = humidity_model.predict(future_humidity)

# ---- Plot Humidity Forecast
plt.figure(figsize=(10, 5))
humidity_model.plot(humidity_forecast, xlabel='Time', ylabel='Humidity (%)')
plt.title('24-Hour Humidity Forecast with Historical Trends')
plt.grid(True)

# ---- Highlight observed data period
plt.fill_between(humidity_forecast['ds'], humidity_forecast['yhat_lower'], humidity_forecast['yhat_upper'], color='lightgreen', alpha=0.2)
plt.annotate('Future Predicted Range', xy=(humidity_forecast['ds'].iloc[-5], humidity_forecast['yhat'].iloc[-5]), color='green')

# ---- Plot the forecast
plt.show()


# ML PREDICTION & ANOMALIES DETECTION | DATA

In [ ]:
'''
* ML PREDICTION & ANOMALIES DETECTION ------------------
*  # Devices:
*     -> DHT22 sensor
*     -> ESP32
*  # Technologies:
*     -> Protocol: HTTP
*     -> Platform: ThingSpeak + Google Colab
*     -> ML Tools: Prediction & Regression - LinearRegression
*  # Activity:
*    -> Using ML for data processing, prediction and anomalies detection
*
*   ANGAZA ELIMU&ALX - IOT SCHOOL: Cohort 1, 2024
* --------------------------------------------------
'''

# Import necessary libraries
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# ThingSpeak Configuration
CHANNEL_ID = userdata.get('CHANNEL_ID')
READ_API_KEY = userdata.get('API_KEY') # Read API Key

# Section 1: Data Collection from ThingSpeak
def fetch_data():
    url = f'https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={READ_API_KEY}&results=1000'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['feeds'])
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['temperature'] = pd.to_numeric(df['field1'], errors='coerce')
    df['humidity'] = pd.to_numeric(df['field2'], errors='coerce')
    print("Data fetched successfully.")
    return df[['created_at', 'temperature', 'humidity']]

# Section 2: Data Preparation
def prepare_data(df):
    df['time_index'] = (df['created_at'] - df['created_at'].min()).dt.total_seconds()
    X = df[['time_index']]
    y_temp = df['temperature']
    y_hum = df['humidity']
    print("Data prepared successfully.")
    return X, y_temp, y_hum

# Section 3: Train Temperature and Humidity Prediction Models
def train_regression_models(X, y_temp, y_hum):
    # Split data for training and testing
    X_train, X_test, y_temp_train, y_temp_test = train_test_split(X, y_temp, test_size=0.2, random_state=42)
    _, _, y_hum_train, y_hum_test = train_test_split(X, y_hum, test_size=0.2, random_state=42)

    # Train temperature prediction model
    temp_model = LinearRegression()
    temp_model.fit(X_train, y_temp_train)
    temp_predictions = temp_model.predict(X_test)
    temp_mse = mean_squared_error(y_temp_test, temp_predictions)

    # Train humidity prediction model
    hum_model = LinearRegression()
    hum_model.fit(X_train, y_hum_train)
    hum_predictions = hum_model.predict(X_test)
    hum_mse = mean_squared_error(y_hum_test, hum_predictions)

    print(f"Temperature Prediction MSE: {temp_mse}")
    print(f"Humidity Prediction MSE: {hum_mse}")
    return temp_model, hum_model

# Section 4.1: Anomaly Detection for Humidity
def humid_anomaly_detection(df):
    hum_mean, hum_std = df['humidity'].mean(), df['humidity'].std()
    threshold_upper = hum_mean + 2 * hum_std
    threshold_lower = hum_mean - 2 * hum_std
    df['humidity_anomaly'] = df['humidity'].apply(lambda x: 'Anomaly' if x > threshold_upper or x < threshold_lower else 'Normal')
    print("Anomaly detection completed.")
    return threshold_upper, threshold_lower

# Section 4.2: Anomaly Detection for Temperature
def temp_anomaly_detection(df):
    temp_mean, temp_std = df['temperature'].mean(), df['temperature'].std()
    threshold_upper = temp_mean + 2 * temp_std
    threshold_lower = temp_mean - 2 * temp_std
    df['temperature_anomaly'] = df['temperature'].apply(lambda x: 'Anomaly' if x > threshold_upper or x < threshold_lower else 'Normal')
    print("Anomaly detection completed.")
    return threshold_upper, threshold_lower

# Section 5: Predict and Test on New Data
def test_new_data(temp_model, hum_model, temp_thresholds, hum_thresholds):
    # Fetch new data
    new_data = fetch_data()
    new_data['time_index'] = (new_data['created_at'] - new_data['created_at'].min()).dt.total_seconds()
    X_new = new_data[['time_index']]

    # Make predictions
    new_data['predicted_temperature'] = temp_model.predict(X_new)
    new_data['predicted_humidity'] = hum_model.predict(X_new)

    # Apply anomaly detection for humidity
    hum_upper, hum_lower = hum_thresholds
    new_data['humidity_anomaly'] = new_data['humidity'].apply(lambda x: 'Anomaly' if x > hum_upper or x < hum_lower else 'Normal')

    # Apply anomaly detection for temperature
    temp_upper, temp_lower = temp_thresholds
    new_data['temperature_anomaly'] = new_data['temperature'].apply(lambda x: 'Anomaly' if x > temp_upper or x < temp_lower else 'Normal')

    print("Predictions and anomaly detection on new data:")
    print(new_data[['created_at', 'temperature', 'predicted_temperature', 'temperature_anomaly',
                    'humidity', 'predicted_humidity', 'humidity_anomaly']].tail())

# Main Execution Flow
if __name__ == "__main__":
    # Step 1: Fetch and prepare the data
    df = fetch_data()
    X, y_temp, y_hum = prepare_data(df)

    # Step 2: Train the ML models for temperature and humidity
    temp_model, hum_model = train_regression_models(X, y_temp, y_hum)

    # Step 3: Set up and apply anomaly detection for humidity and temperature
    temp_thresholds = temp_anomaly_detection(df)
    hum_thresholds = humid_anomaly_detection(df)

    # Step 4: Test and use the models on new data
    test_new_data(temp_model, hum_model, temp_thresholds, hum_thresholds)
